In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 3
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC50(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC50(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC50(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料


D:\Hu_code\ResNet18_ESC50\ESC50_kFold_135s\..\TrainDataPreprocessing.py:57: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
Data Loading Time: 133.47 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        
        #------------------------------------------------
        #需要調整的地方
        #------------------------------------------------
        #--要用幾秒訓練 data_1S/data_3S/data_1S
        #--學習率
        #--epoch
        if(TL==0):
            trainData = data_1S
            epoch = 90
            lr = 0.0032
        elif(TL==1):
            trainData = data_3S
            epoch = 90
            lr = 0.0032
        else:
            trainData = data_5S
            epoch = 90
            lr = 0.0032
        #------------------------------------------------
    

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型
        model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold,
                                                                setLR=lr,
                                                                setEpoch=epoch)
    
        #測試準確度
        test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
        test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
        test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/90 Train_Loss: 2.6301379203796387 Train_acc: 13.937501%
Train Epoch: 2/90 Train_Loss: 1.7156926393508911 Train_acc: 32.000000%
Train Epoch: 3/90 Train_Loss: 1.8461285829544067 Train_acc: 45.250004%
Train Epoch: 4/90 Train_Loss: 1.4303910732269287 Train_acc: 55.350002%
Train Epoch: 5/90 Train_Loss: 1.1439127922058105 Train_acc: 64.187500%
Train Epoch: 6/90 Train_Loss: 0.9470304250717163 Train_acc: 69.575005%
Train Epoch: 7/90 Train_Loss: 0.8457344770431519 Train_acc: 75.600006%
Train Epoch: 8/90 Train_Loss: 0.7075022459030151 Train_acc: 77.762505%
Train Epoch: 9/90 Train_Loss: 0.6924932599067688 Train_acc: 81.237503%
Train Epoch: 10/90 Train_Loss: 0.5655866861343384 Train_acc: 83.900002%
Train Epoch: 11/90 Train_Loss: 0.

Train Epoch: 16/90 Train_Loss: 0.08598456531763077 Train_acc: 95.895836%
Train Epoch: 17/90 Train_Loss: 0.1834755390882492 Train_acc: 97.187500%
Train Epoch: 18/90 Train_Loss: 0.12261578440666199 Train_acc: 97.395836%
Train Epoch: 19/90 Train_Loss: 0.284455806016922 Train_acc: 97.333336%
Train Epoch: 20/90 Train_Loss: 0.006315592210739851 Train_acc: 97.416672%
Train Epoch: 21/90 Train_Loss: 0.08327538520097733 Train_acc: 97.416672%
Train Epoch: 22/90 Train_Loss: 0.08152934908866882 Train_acc: 97.416672%
Train Epoch: 23/90 Train_Loss: 0.07041653990745544 Train_acc: 97.416672%
Train Epoch: 24/90 Train_Loss: 0.12975235283374786 Train_acc: 97.416672%
Train Epoch: 25/90 Train_Loss: 0.14869174361228943 Train_acc: 97.416672%
Train Epoch: 26/90 Train_Loss: 0.001035352237522602 Train_acc: 97.437500%
Train Epoch: 27/90 Train_Loss: 0.04920987784862518 Train_acc: 97.416672%
Train Epoch: 28/90 Train_Loss: 0.06789219379425049 Train_acc: 97.395836%
Train Epoch: 29/90 Train_Loss: 0.17494511604309082 T

Train Epoch: 33/90 Train_Loss: 6.458308052970096e-05 Train_acc: 100.000000%
Train Epoch: 34/90 Train_Loss: 0.00012586868251673877 Train_acc: 100.000000%
Train Epoch: 35/90 Train_Loss: 0.00029479991644620895 Train_acc: 100.000000%
Train Epoch: 36/90 Train_Loss: 0.00015949438966345042 Train_acc: 100.000000%
Train Epoch: 37/90 Train_Loss: 0.000265625974861905 Train_acc: 100.000000%
Train Epoch: 38/90 Train_Loss: 0.00012372047058306634 Train_acc: 100.000000%
Train Epoch: 39/90 Train_Loss: 0.00010777202987810597 Train_acc: 100.000000%
Train Epoch: 40/90 Train_Loss: 0.00010992064926540479 Train_acc: 100.000000%
Train Epoch: 41/90 Train_Loss: 8.608600910520181e-05 Train_acc: 100.000000%
Train Epoch: 42/90 Train_Loss: 0.00013204246351961046 Train_acc: 100.000000%
Train Epoch: 43/90 Train_Loss: 9.028655767906457e-05 Train_acc: 100.000000%
Train Epoch: 44/90 Train_Loss: 3.315759022370912e-05 Train_acc: 100.000000%
Train Epoch: 45/90 Train_Loss: 5.410595622379333e-05 Train_acc: 100.000000%
Train 

Train Epoch: 48/90 Train_Loss: 0.22653479874134064 Train_acc: 93.112503%
Train Epoch: 49/90 Train_Loss: 0.15152941644191742 Train_acc: 93.100006%
Train Epoch: 50/90 Train_Loss: 0.1872839331626892 Train_acc: 93.125008%
Train Epoch: 51/90 Train_Loss: 0.16874189674854279 Train_acc: 93.100006%
Train Epoch: 52/90 Train_Loss: 0.19531278312206268 Train_acc: 93.112503%
Train Epoch: 53/90 Train_Loss: 0.3005591630935669 Train_acc: 93.125008%
Train Epoch: 54/90 Train_Loss: 0.039303794503211975 Train_acc: 93.112503%
Train Epoch: 55/90 Train_Loss: 0.15470141172409058 Train_acc: 93.125008%
Train Epoch: 56/90 Train_Loss: 0.02859366126358509 Train_acc: 93.112503%
Train Epoch: 57/90 Train_Loss: 0.2321319282054901 Train_acc: 93.112503%
Train Epoch: 58/90 Train_Loss: 0.22266705334186554 Train_acc: 93.112503%
Train Epoch: 59/90 Train_Loss: 0.13722431659698486 Train_acc: 93.125008%
Train Epoch: 60/90 Train_Loss: 0.2980712056159973 Train_acc: 93.125008%
Train Epoch: 61/90 Train_Loss: 0.17008957266807556 Tra

Train Epoch: 65/90 Train_Loss: 0.11087481677532196 Train_acc: 97.395836%
Train Epoch: 66/90 Train_Loss: 0.025298941880464554 Train_acc: 97.395836%
Train Epoch: 67/90 Train_Loss: 0.1647583395242691 Train_acc: 97.395836%
Train Epoch: 68/90 Train_Loss: 0.09261539578437805 Train_acc: 97.395836%
Train Epoch: 69/90 Train_Loss: 0.07237949967384338 Train_acc: 97.395836%
Train Epoch: 70/90 Train_Loss: 0.0753929615020752 Train_acc: 97.395836%
Train Epoch: 71/90 Train_Loss: 0.12848259508609772 Train_acc: 97.395836%
Train Epoch: 72/90 Train_Loss: 0.07952319830656052 Train_acc: 97.395836%
Train Epoch: 73/90 Train_Loss: 0.10987116396427155 Train_acc: 97.395836%
Train Epoch: 74/90 Train_Loss: 0.26252755522727966 Train_acc: 97.395836%
Train Epoch: 75/90 Train_Loss: 0.0668378621339798 Train_acc: 97.395836%
Train Epoch: 76/90 Train_Loss: 0.06630577147006989 Train_acc: 97.395836%
Train Epoch: 77/90 Train_Loss: 0.0804833322763443 Train_acc: 97.395836%
Train Epoch: 78/90 Train_Loss: 0.08800944685935974 Tra

...

Train Epoch: 33/90 Train_Loss: 0.42538419365882874 Train_acc: 92.612503%
Train Epoch: 34/90 Train_Loss: 0.41369253396987915 Train_acc: 92.625008%
Train Epoch: 35/90 Train_Loss: 0.2595445215702057 Train_acc: 92.600006%
Train Epoch: 36/90 Train_Loss: 0.3925780653953552 Train_acc: 92.662506%
Train Epoch: 37/90 Train_Loss: 0.2431468665599823 Train_acc: 92.650002%
Train Epoch: 38/90 Train_Loss: 0.5487125515937805 Train_acc: 92.575005%
Train Epoch: 39/90 Train_Loss: 0.2621815502643585 Train_acc: 92.675003%
Train Epoch: 40/90 Train_Loss: 0.25071731209754944 Train_acc: 92.637505%
Train Epoch: 41/90 Train_Loss: 0.21334920823574066 Train_acc: 92.612503%
Train Epoch: 42/90 Train_Loss: 0.19540590047836304 Train_acc: 92.650002%
Train Epoch: 43/90 Train_Loss: 0.41109147667884827 Train_acc: 92.662506%
Train Epoch: 44/90 Train_Loss: 0.13173677027225494 Train_acc: 92.675003%
Train Epoch: 45/90 Train_Loss: 0.13625621795654297 Train_acc: 92.650002%
Train Epoch: 46/90 Train_Loss: 0.18192680180072784 Train

Train Epoch: 51/90 Train_Loss: 0.15927395224571228 Train_acc: 97.208336%
Train Epoch: 52/90 Train_Loss: 0.05883706361055374 Train_acc: 97.208336%
Train Epoch: 53/90 Train_Loss: 0.06557637453079224 Train_acc: 97.208336%
Train Epoch: 54/90 Train_Loss: 0.0769859254360199 Train_acc: 97.187500%
Train Epoch: 55/90 Train_Loss: 0.1826135665178299 Train_acc: 97.208336%
Train Epoch: 56/90 Train_Loss: 0.0004245831514708698 Train_acc: 97.208336%
Train Epoch: 57/90 Train_Loss: 0.07134684175252914 Train_acc: 97.208336%
Train Epoch: 58/90 Train_Loss: 0.2228364646434784 Train_acc: 97.187500%
Train Epoch: 59/90 Train_Loss: 0.18059644103050232 Train_acc: 97.208336%
Train Epoch: 60/90 Train_Loss: 0.10989098995923996 Train_acc: 97.187500%
Train Epoch: 61/90 Train_Loss: 9.311683243140578e-05 Train_acc: 97.187500%
Train Epoch: 62/90 Train_Loss: 0.14348028600215912 Train_acc: 97.229172%
Train Epoch: 63/90 Train_Loss: 0.05682677403092384 Train_acc: 97.187500%
Train Epoch: 64/90 Train_Loss: 0.04332034289836883

Train Epoch: 66/90 Train_Loss: 7.197460217867047e-05 Train_acc: 100.000000%
Train Epoch: 67/90 Train_Loss: 5.5502467148471624e-05 Train_acc: 100.000000%
Train Epoch: 68/90 Train_Loss: 8.941383566707373e-05 Train_acc: 100.000000%
Train Epoch: 69/90 Train_Loss: 0.0001733093522489071 Train_acc: 100.000000%
Train Epoch: 70/90 Train_Loss: 0.00010153129551326856 Train_acc: 100.000000%
Train Epoch: 71/90 Train_Loss: 0.00018135410209652036 Train_acc: 100.000000%
Train Epoch: 72/90 Train_Loss: 0.00010494266462046653 Train_acc: 100.000000%
Train Epoch: 73/90 Train_Loss: 0.0001944520918186754 Train_acc: 100.000000%
Train Epoch: 74/90 Train_Loss: 0.00022460926265921444 Train_acc: 100.000000%
Train Epoch: 75/90 Train_Loss: 7.457208266714588e-05 Train_acc: 100.000000%
Train Epoch: 76/90 Train_Loss: 0.00010018159809987992 Train_acc: 100.000000%
Train Epoch: 77/90 Train_Loss: 5.8907917264150456e-05 Train_acc: 100.000000%
Train Epoch: 78/90 Train_Loss: 3.362621646374464e-05 Train_acc: 100.000000%
Train

Train Epoch: 83/90 Train_Loss: 0.26813000440597534 Train_acc: 92.862503%
Train Epoch: 84/90 Train_Loss: 0.27619341015815735 Train_acc: 92.875008%
Train Epoch: 85/90 Train_Loss: 0.3447132408618927 Train_acc: 92.862503%
Train Epoch: 86/90 Train_Loss: 0.3516461253166199 Train_acc: 92.862503%
Train Epoch: 87/90 Train_Loss: 0.18742145597934723 Train_acc: 92.862503%
Train Epoch: 88/90 Train_Loss: 0.1544816792011261 Train_acc: 92.875008%
Train Epoch: 89/90 Train_Loss: 0.28246167302131653 Train_acc: 92.850006%
Train Epoch: 90/90 Train_Loss: 0.26193293929100037 Train_acc: 92.875008%
Test accuracy of the model: 50.200000%
Test accuracy of the model: 52.833333%
Test accuracy of the model: 53.500000%
Training Time: 287.90 seconds
----------------------------------------------------------------------------------------------------------------
fold:  5 TL:  1
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/90 Train_Loss: 

Train Epoch: 5/90 Train_Loss: 0.029102468863129616 Train_acc: 97.125000%
Train Epoch: 6/90 Train_Loss: 0.006388003006577492 Train_acc: 98.375000%
Train Epoch: 7/90 Train_Loss: 0.07428603619337082 Train_acc: 98.312500%
Train Epoch: 8/90 Train_Loss: 0.0924970954656601 Train_acc: 98.437500%
Train Epoch: 9/90 Train_Loss: 0.004417381249368191 Train_acc: 98.500000%
Train Epoch: 10/90 Train_Loss: 0.013990959152579308 Train_acc: 99.250000%
Train Epoch: 11/90 Train_Loss: 0.012429116293787956 Train_acc: 99.312500%
Train Epoch: 12/90 Train_Loss: 0.0010742079466581345 Train_acc: 99.687500%
Train Epoch: 13/90 Train_Loss: 0.003308819141238928 Train_acc: 99.875000%
Train Epoch: 14/90 Train_Loss: 0.00040167904808185995 Train_acc: 100.000000%
Train Epoch: 15/90 Train_Loss: 0.0004162994446232915 Train_acc: 100.000000%
Train Epoch: 16/90 Train_Loss: 0.009280035272240639 Train_acc: 99.937500%
Train Epoch: 17/90 Train_Loss: 9.291304741054773e-05 Train_acc: 100.000000%
Train Epoch: 18/90 Train_Loss: 0.00012

In [5]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()